# Designing Composites with Optimal Mechanical Properties

## Introduction

Composite materials are advanced materials engineered by combining two or more distinct constituent materials, each with significantly different physical or chemical properties. The resulting composite exhibits unique characteristics that differ from those of its individual components, while the constituents remain separate and distinct within the final structure. 

The objective of this lab is to leverage **NumPy** to develop a robust system for calculating the properties of a composite material based on the properties of its individual components, their volume fractions, and the orientation of fibers. This capability is crucial for designing high-performance materials and structures. For example, carbon fiber composites are widely used in aerospace applications due to their exceptional strength-to-weight ratio. Accurate prediction of the properties of these materials is vital to ensure they meet application-specific requirements and to design materials with tailored properties along different axes.

## Goals: 

You want to compute the **effective stiffness** of a 2D composite (plane stress) reinforced by short fibers (or platelets) whose orientations are **not uniform**—instead, they follow some **anisotropic orientation distribution**. Each filler phase has **local** stiffness $\mathbf{Q}_\text{local}$ in its principal axes, and you also know the **matrix** stiffness. Your goal is to:

1. **Define** the 2D plane-stress stiffness matrix $\mathbf{Q}_\text{local}$ for the **filler** in its local coordinate system (aligned with principal directions).
2. **Rotate** that local stiffness into the **global** coordinates for each orientation $\theta$ the filler may have, using a transformation matrix $\mathbf{T}(\theta)$.
3. **Combine** these rotated filler stiffness matrices, weighted by their **orientation distribution** (i.e., the fraction of fibers oriented at each $\theta$), and then add the **matrix** contribution to get the overall **effective** stiffness of the composite.


### Your tasks are to **Write a Python/NumPy script** that:
   - Builds a **local filler** stiffness matrix $\mathbf{Q}_\text{local}$.
   - Implements **rotation** of $\mathbf{Q}_\text{local}$ to $\mathbf{\bar{Q}}(\theta)$ for a given $\theta$.
   - Summation or integration of $\mathbf{\bar{Q}}(\theta)$ across the **anisotropic distribution** to get $\mathbf{Q}_\text{filler,eff}$.
   - Combines the filler result with **matrix** stiffness to obtain the final $\mathbf{Q}_\text{eff}$.


**Don't Worry** you are not expected to know the math required to solve this problem, we will guide you through the process, and explain the math behind the solution as we go along.

## Part 1: Matrix Stiffness

First, we want to define the 2D plane stiffness matrix. The stiffness matrix $\mathbf{Q}$ provides the stiffness of the material along all axis in the material coordinate system. The stiffness matrix is a 3x3 matrix for 2D plane stress problems.

$$
\mathbf{Q}_\text{local} = 
\begin{bmatrix}
Q_{11} & Q_{12} & 0 \\
Q_{12} & Q_{22} & 0 \\
0      & 0      & G_{12}
\end{bmatrix}.
$$

- **$Q_{11}$**:  
  This is the stiffness in the **1-direction** (typically aligned with the fiber's principal axis). It quantifies the material's resistance to normal strain along the 1-direction when subjected to normal stress in the same direction.

- **$Q_{22}$**:  
  This is the stiffness in the **2-direction** (perpendicular to the 1-direction in the plane). It quantifies the material's resistance to normal strain along the 2-direction when subjected to normal stress in the same direction.

- **$Q_{12}$**:  
  This represents the coupling stiffness between the 1- and 2-directions. It arises from the material’s anisotropy, where normal stress in one direction may cause strain in the perpendicular direction.

- **$G_{12}$**:  
  This is the **in-plane shear modulus**, which quantifies the material's resistance to in-plane shear deformation. It is a measure of the stiffness associated with shearing action in the plane of the composite.

- **0 entries**:  
  The zero entries indicate that there is no coupling between shear deformation and normal stresses (in-plane shear stresses do not cause normal strains and vice versa) in the principal axes. This is a characteristic of the orthotropic behavior assumed in this matrix.  

In this case we will assume that the matrix stiffness is isotropic, meaning that the stiffness is the same in all directions.

### Python Implementation:

We will begin by implementing the local filler stiffness matrix $\mathbf{Q}_\text{local}$ in Python using NumPy. To achieve this, we define a function `local_stiffness_matrix` that takes the following parameters as input:

- Elastic moduli: $E_1$ and $E_2$,
- Poisson’s ratio: $\nu_{12}$,  
- Shear modulus: $G_{12}$.

The function will return the local stiffness matrix $\mathbf{Q}_\text{local}$. We have provided the scaffolding for the function, and you need to fill in the missing code.

1. compute the Poisson's ratio in the transverse direction, denoted as $\nu_{21}$ to the variable `v21`, based on the given Poisson's ratio in the longitudinal direction ($\nu_{12}$) and the elastic moduli ($E_1$ and $E_2$), using the formula:
$$ \nu_{21} = \frac{E_2}{E_1} \cdot \nu_{12} $$

2. Compute the denominator factor `denom` which is used to compute the stiffness matrix components, using the formula:
$$ denom = 1 - \nu_{12} \cdot \nu_{21} $$

3. Compute the stiffness matrix components $Q_{11}$, $Q_{22}$, and $Q_{12}$ and assign them to the variables Q11, Q22, and Q12 respectively using the formulas:
$$ Q_{11} = \frac{E_1}{1 - \nu_{12} \cdot \nu_{21}} $$
$$ Q_{22} = \frac{E_2}{1 - \nu_{12} \cdot \nu_{21}} $$
$$ Q_{12} = \frac{\nu_{12} \cdot E_2}{1 - \nu_{12} \cdot \nu_{21}} $$

4. Construct the local stiffness matrix as a 2D NumPy array `Q` using the computed stiffness matrix components $Q_{11}$, $Q_{22}$, and $Q_{12}$, per the matrix definition provided:

$$
\mathbf{Q}_\text{local} = 
\begin{bmatrix}
Q_{11} & Q_{12} & 0 \\
Q_{12} & Q_{22} & 0 \\
0      & 0      & G_{12}
\end{bmatrix}.
$$

5. Compute the matrix stiffness tensor:

Use the `local_stiffness_orthotropic` function to compute the local stiffness matrix for the filler phase with the following properties:

- Elastic modulus in the 1-direction: $E_1 = E_2 = 2$ GPa
- Poisson's ratio in the 1-direction: $\nu_{12} = 0.33$
- Shear modulus: $G_{12} = 0.8$ GPa

Assign this to the variable `Q_matrix`.


In [ ]:
import numpy as np

def local_stiffness_orthotropic(E1, E2, v12, G12, otter=False):
    
    # 1. Compute the transverse Poisson's ratio
    # BEGIN SOLUTION
    v21 = (E2 / E1) * v12
    # END SOLUTION
    
    # 2. Compute the denominator of the stiffness matrix Q
    # BEGIN SOLUTION
    denom = 1.0 - v12 * v21
    # END SOLUTION
    
    # 3. Compute the stiffness matrix Q
    # BEGIN SOLUTION
    Q11 = E1 / denom
    Q22 = E2 / denom
    Q12 = v12 * E2 / denom
    # END SOLUTION

    # 4. Assemble the stiffness matrix Q
    # BEGIN SOLUTION
    Q = np.array([
        [Q11, Q12,  0.0],
        [Q12, Q22,  0.0],
        [0.0,  0.0, G12]
    ])
    # END SOLUTION
    if otter:
        return Q, E1, E2, v12, G12, v21, denom, Q11, Q22, Q12
    return Q

# BEGIN SOLUTION
Q_matrix = local_stiffness_orthotropic(2, 2, 0.33, .8)
# END SOLUTION

In [ ]:
""" # BEGIN TEST CONFIG
points: 1
hidden: false
success_message: "Success: Output Q is a 3x3 matrix!"
failure_message: "Failed: Output Q is not 3x3!"
log_variables: ["Q_shape"]
"""  # END TEST CONFIG

import numpy as np

result = local_stiffness_orthotropic(2, 2, 0.33, 0.8, otter=True)

# 3) Test shape of Q
Q, E1_ret, E2_ret, v12_ret, G12_ret, v21_ret, denom_ret, Q11_ret, Q22_ret, Q12_ret = (
    result
)
Q_shape = Q.shape
assert Q_shape == (3, 3), f"Expected Q to be 3x3, but got {Q_shape}."

In [ ]:
""" # BEGIN TEST CONFIG
points: 1
hidden: false
success_message: "Success: v21 is computed correctly!"
failure_message: "Failed: v21 is incorrect!"
log_variables: ["v21_ret","v21_exp"]
"""  # END TEST CONFIG

import numpy as np

result = local_stiffness_orthotropic(2, 2, 0.33, 0.8, otter=True)

# 3) Test shape of Q
Q, E1_ret, E2_ret, v12_ret, G12_ret, v21_ret, denom_ret, Q11_ret, Q22_ret, Q12_ret = (
    result
)

# 8) Test v21
v21_exp = 0.33
assert np.isclose(
    v21_ret, v21_exp, atol=1e-6
), f"Expected v21={v21_exp}, but got {v21_ret}."

In [ ]:
""" # BEGIN TEST CONFIG
points: 1
hidden: false
success_message: "Success: denom is computed correctly!"
failure_message: "Failed: denom is incorrect!"
log_variables: ["denom_ret","denom_exp"]
"""  # END TEST CONFIG

import numpy as np

result = local_stiffness_orthotropic(2, 2, 0.33, 0.8, otter=True)

# 3) Test shape of Q
Q, E1_ret, E2_ret, v12_ret, G12_ret, v21_ret, denom_ret, Q11_ret, Q22_ret, Q12_ret = (
    result
)

# 9) Test denom
denom_exp =  0.8911
assert np.isclose(
    denom_ret, denom_exp, atol=1e-6
), f"Expected denom={denom_exp}, but got {denom_ret}."

In [ ]:
""" # BEGIN TEST CONFIG
points: 1
hidden: false
success_message: "Success: Q11 is computed correctly!"
failure_message: "Failed: Q11 is incorrect!"
log_variables: ["Q11_ret","Q11_exp"]
"""  # END TEST CONFIG

import numpy as np

result = local_stiffness_orthotropic(2, 2, 0.33, 0.8, otter=True)

# 3) Test shape of Q
Q, E1_ret, E2_ret, v12_ret, G12_ret, v21_ret, denom_ret, Q11_ret, Q22_ret, Q12_ret = (
    result
)

# 10) Test Q11
Q11_exp = 2 / 0.8911
assert np.isclose(
    Q11_ret, Q11_exp, atol=1e-6
), f"Expected Q11={Q11_exp}, but got {Q11_ret}."

In [ ]:
""" # BEGIN TEST CONFIG
points: 1
hidden: false
success_message: "Success: Q22 is computed correctly!"
failure_message: "Failed: Q22 is incorrect!"
log_variables: ["Q22_ret","Q22_exp"]
"""  # END TEST CONFIG

import numpy as np

result = local_stiffness_orthotropic(2, 2, 0.33, 0.8, otter=True)

# 3) Test shape of Q
Q, E1_ret, E2_ret, v12_ret, G12_ret, v21_ret, denom_ret, Q11_ret, Q22_ret, Q12_ret = (
    result
)

# 11) Test Q22
Q22_exp = 2 / 0.8911
assert np.isclose(
    Q22_ret, Q22_exp, atol=1e-6
), f"Expected Q22={Q22_exp}, but got {Q22_ret}."

In [ ]:
""" # BEGIN TEST CONFIG
points: 1
hidden: false
success_message: "Success: Q12 is computed correctly!"
failure_message: "Failed: Q12 is incorrect!"
log_variables: ["Q12_ret","Q12_exp"]
"""  # END TEST CONFIG

import numpy as np

result = local_stiffness_orthotropic(2, 2, 0.33, 0.8, otter=True)

# 3) Test shape of Q
Q, E1_ret, E2_ret, v12_ret, G12_ret, v21_ret, denom_ret, Q11_ret, Q22_ret, Q12_ret = (
    result
)

# 12) Test Q12
Q12_exp = (0.33 * 2) / 0.8911
assert np.isclose(
    Q12_ret, Q12_exp, atol=1e-6
), f"Expected Q12={Q12_exp}, but got {Q12_ret}."

In [ ]:
""" # BEGIN TEST CONFIG
points: 4
hidden: false
success_message: "Success: Full Q matrix test passed!"
failure_message: "Failed: The Q matrix does not match the expected orthotropic stiffness."
log_variables: ["Q","Q_expected"]
"""  # END TEST CONFIG

import numpy as np

result = local_stiffness_orthotropic(2, 2, 0.33, 0.8, otter=True)

# 3) Test shape of Q
Q, E1_ret, E2_ret, v12_ret, G12_ret, v21_ret, denom_ret, Q11_ret, Q22_ret, Q12_ret = (
    result
)

# Expected Q from the formula:
Q_expected = np.array(
    [[2.24441701, 0.74065761, 0.0], [0.74065761, 2.24441701, 0.0], [0.0, 0.0, 0.8]]
)

assert Q.shape == (3, 3), f"Expected Q to be 3x3, but got {Q.shape}"
assert np.allclose(Q, Q_expected, atol=1e-6), f"Expected:\n{Q_expected}\nBut got:\n{Q}"

## Part 2: Rotation Matrix to Transform from Local to Global Coordinates

### Context

In 2D plane stress problems, we typically work with stress components ($\sigma_x, \sigma_y, \tau_{xy}$) or strain components ($\epsilon_x, \epsilon_y, \gamma_{xy}$) defined in the local or global coordinate system. These components can be transformed between coordinate systems using a transformation matrix ($\mathbf{T}(\theta)$), where $\theta$ represents the angle of rotation between the local and global axes.

The transformation matrix ($\mathbf{T}(\theta)$) is derived based on trigonometric relationships in the rotated coordinate system. The components of the transformation matrix account for how normal and shear stresses/strains change under rotation.

For engineering applications like composite materials or anisotropic elasticity, it is often necessary to compute the stiffness matrix ($\mathbf{\bar{Q}}$) in the **global coordinate system** from the stiffness matrix ($\mathbf{Q}_\text{local}$) in the **local coordinate system**. This transformation involves both the transformation matrix and its transpose/inverse, as shown below.


#### Stress/Strain Representation in Plane Stress

$$
\text{Stress components: } \sigma_x, \sigma_y, \tau_{xy}
$$

$$
\text{Strain components: } \epsilon_x, \epsilon_y, \gamma_{xy}
$$


#### Transformation Matrix (3×3)

$$
\mathbf{T}(\theta) = 
\begin{bmatrix}
\cos^2\theta & \sin^2\theta & 2\sin\theta \cos\theta \\
\sin^2\theta & \cos^2\theta & -2\sin\theta \cos\theta \\
-\sin\theta \cos\theta & \sin\theta \cos\theta & \cos^2\theta - \sin^2\theta
\end{bmatrix}.
$$


#### Stiffness Transformation

$$
\mathbf{\bar{Q}}(\theta) = \mathbf{T}^{-1}(\theta) \; \mathbf{Q}_\text{local} \; \mathbf{T}^{-T}(\theta)
$$

where:

$
\mathbf{\bar{Q}}(\theta) \text{ is the global stiffness matrix.}
$

$
\mathbf{Q}_\text{local} \text{ is the stiffness matrix in the local coordinate system.}
$

$
\mathbf{T}(\theta) \text{ is the transformation matrix.}
$

$
\mathbf{T}^{-1} \text{ and } \mathbf{T}^{-T} \text{ are the inverse and transpose-inverse of } \mathbf{T}(\theta), \text{ respectively.}
$


### Python Implementation:

#### Part 2.1: Rotation Matrix

We will implement a function `rotation_matrix` that computes the transformation matrix $\mathbf{T}(\theta)$ for a given angle $\theta$. The function will take the angle $\theta$ as input and return the transformation matrix $\mathbf{T}(\theta)$.

1. Compute the trigonometric functions $\sin\theta$ and $\cos\theta$ using NumPy's `sin` and `cos` functions, respectively. Assign the results to the variables `sin_theta` and `cos_theta`.
2. Construct the transformation matrix $\mathbf{T}(\theta)$ as a 2D NumPy array `T` using the computed trigonometric functions $\sin\theta$ and $\cos\theta$, per the matrix definition provided:

$$
\mathbf{T}(\theta) = 
\begin{bmatrix}
\cos^2\theta & \sin^2\theta & 2\sin\theta \cos\theta \\
\sin^2\theta & \cos^2\theta & -2\sin\theta \cos\theta \\
-\sin\theta \cos\theta & \sin\theta \cos\theta & \cos^2\theta - \sin^2\theta
\end{bmatrix}.
$$

In [ ]:
def transformation_matrix_2D(theta, otter=False):
    """
    Build the 3x3 transformation matrix for plane-stress (sigma_x, sigma_y, tau_xy).
    """
    # 1. Compute the cosine and sine of the angle theta
    # BEGIN SOLUTION
    cos_theta = np.cos(theta)
    sin_theta = np.sin(theta)
    # END SOLUTION
    
    # 2. Build the transformation matrix
    # BEGIN SOLUTION
    T = np.array(
        [
            [cos_theta * cos_theta, sin_theta * sin_theta, 2 * sin_theta * cos_theta],
            [sin_theta * sin_theta, cos_theta * cos_theta, -2 * sin_theta * cos_theta],
            [
                -sin_theta * cos_theta,
                sin_theta * cos_theta,
                cos_theta * cos_theta - sin_theta * sin_theta,
            ],
        ]
    )
    # END SOLUTION
    
    if otter:
        return T, cos_theta, sin_theta
    return T

In [ ]:
""" # BEGIN TEST CONFIG
points: 1
hidden: false
success_message: "Success: T is a 3x3 matrix!"
failure_message: "Failed: T is not 3x3!"
log_variables: ["T_shape"]
"""  # END TEST CONFIG

import numpy as np

# Test (3): Check the shape of T
theta = 0.0
T = transformation_matrix_2D(theta)  # default otter=False
T_shape = T.shape
assert T_shape == (3, 3), f"Expected T to be 3x3, but got {T_shape}."

In [ ]:
""" # BEGIN TEST CONFIG
points: 1
hidden: false
success_message: "Success: transformation_matrix_2D(0.0) is correct!"
failure_message: "Failed: transformation_matrix_2D(0.0) is incorrect!"
log_variables: ["T","T_expected"]
"""  # END TEST CONFIG

import numpy as np

# Test (4): At theta=0, cos(0) = 1, sin(0) = 0
# T should be:
# [ [1, 0, 0],
#   [0, 1, 0],
#   [0, 0, 1] ]
T_expected = np.array([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]], dtype=float)

theta = 0.0
T = transformation_matrix_2D(theta)
assert np.allclose(T, T_expected, atol=1e-9), f"Expected:\n{T_expected}\nGot:\n{T}"

In [ ]:
""" # BEGIN TEST CONFIG
points: 1
hidden: false
success_message: "Success: transformation_matrix_2D(pi/2) is correct!"
failure_message: "Failed: transformation_matrix_2D(pi/2) is incorrect!"
log_variables: ["T","T_expected"]
"""  # END TEST CONFIG

import numpy as np
import math

# Test (5): For theta = pi/2
# cos(pi/2) = 0, sin(pi/2) = 1
# T should be:
# [
#   [0*0,   1*1,   2*1*0]    = [0, 1, 0],
#   [1*1,   0*0,   -2*1*0]   = [1, 0, 0],
#   [-1*0,  1*0,   0*0 - 1*1]= [0,  0, -1]
# ]
T_expected = np.array([[0.0, 1.0, 0.0], [1.0, 0.0, 0.0], [0.0, 0.0, -1.0]], dtype=float)

theta = math.pi / 2
T = transformation_matrix_2D(theta)
assert np.allclose(T, T_expected, atol=1e-9), f"Expected:\n{T_expected}\nGot:\n{T}"

In [ ]:
""" # BEGIN TEST CONFIG
points: 1
hidden: false
success_message: "Success: transformation_matrix_2D(pi) is correct!"
failure_message: "Failed: transformation_matrix_2D(pi) is incorrect!"
log_variables: ["T","T_expected"]
"""  # END TEST CONFIG

import numpy as np
import math

# Test (6): For theta = pi
# cos(pi) = -1, sin(pi) = 0
# T = [
#   [(-1)^2, (0)^2, 2*0*(-1)]   = [1, 0, 0],
#   [(0)^2,  (-1)^2, -2*0*(-1)] = [0, 1, 0],
#   [-0*(-1),  0*(-1), (-1)^2 - 0^2 ] = [0, 0, 1]
# ]
T_expected = np.array([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]], dtype=float)

theta = math.pi
T = transformation_matrix_2D(theta)
assert np.allclose(T, T_expected, atol=1e-9), f"Expected:\n{T_expected}\nGot:\n{T}"

In [ ]:
""" # BEGIN TEST CONFIG
points: 3
hidden: false
success_message: "Success: transformation_matrix_2D with otter=True returns (T, cos_theta, sin_theta) correctly!"
failure_message: "Failed: The returned cos_theta or sin_theta is incorrect!"
log_variables: ["cos_theta","sin_theta","cos_expected","sin_expected"]
"""  # END TEST CONFIG

import numpy as np
import math

# Test (7): Check cos_theta and sin_theta returned with otter=True
theta = math.radians(30)  # 30 degrees
# cos(30 deg) ~ 0.8660254038, sin(30 deg) ~ 0.5
T, cos_theta, sin_theta = transformation_matrix_2D(theta, otter=True)

cos_expected = math.cos(theta)
sin_expected = math.sin(theta)

assert np.isclose(
    cos_theta, cos_expected, atol=1e-9
), f"Expected cos={cos_expected}, got {cos_theta}"
assert np.isclose(
    sin_theta, sin_expected, atol=1e-9
), f"Expected sin={sin_expected}, got {sin_theta}"

In [ ]:
""" # BEGIN TEST CONFIG
points: 2
hidden: false
success_message: "Success: transformation_matrix_2D(30 degrees) is correct!"
failure_message: "Failed: transformation_matrix_2D(30 degrees) is incorrect!"
log_variables: ["T","T_expected"]
"""  # END TEST CONFIG

import numpy as np
import math

# Test (8): For 30 degrees (pi/6)
theta = math.pi / 6
cos_th = math.cos(theta)  # ~ 0.8660254038
sin_th = math.sin(theta)  # ~ 0.5

# Build expected T manually:
# T[0,0] = cos^2(30) = 0.8660254^2
# T[0,1] = sin^2(30) = 0.5^2
# T[0,2] = 2 * sin(30)*cos(30) = 2*0.5*0.8660254
# T[1,0] = sin^2(30)
# T[1,1] = cos^2(30)
# T[1,2] = -2 * sin(30)*cos(30)
# T[2,0] = -sin(30)*cos(30)
# T[2,1] = sin(30)*cos(30)
# T[2,2] = cos^2(30) - sin^2(30)

T_expected = np.array(
    [
        [cos_th**2, sin_th**2, 2 * cos_th * sin_th],
        [sin_th**2, cos_th**2, -2 * cos_th * sin_th],
        [-cos_th * sin_th, cos_th * sin_th, cos_th**2 - sin_th**2],
    ],
    dtype=float,
)

T = transformation_matrix_2D(theta)
assert np.allclose(T, T_expected, atol=1e-9), f"Expected:\n{T_expected}\nGot:\n{T}"

#### Part 2.2: Transformation of Stiffness Matrix

Next, we will implement a function `transform_stiffness` that computes the transformed stiffness matrix $\mathbf{\bar{Q}}(\theta)$ in the global coordinate system for a given local stiffness matrix $\mathbf{Q}_\text{local}$ and angle $\theta$. The function will take the local stiffness matrix $\mathbf{Q}_\text{local}$ and angle $\theta$ as inputs and return the transformed stiffness matrix $\mathbf{\bar{Q}}(\theta)$.

1. Compute the transformation matrix $\mathbf{T}(\theta)$ using the `rotation_matrix` function with the input angle $\theta$. To do this, call the `rotation_matrix` function with the input angle $\theta$ and assign the result to the variable `T`. You can call the function you wrote just like any function you import from a library.

2. Compute the inverse of the transformation matrix $\mathbf{T}^{-1}(\theta)$ using NumPy's `linalg.inv` function. Assign the result to the variable `T_inv`.

3. Compute the transpose-inverse of the transformation matrix $\mathbf{T}^{-T}(\theta)$ using NumPy's `transpose` built-in method. This can be easily done using the <2D Numpy Array>.T method. Assign the result to the variable `T_inv_transpose`.

4. Compute the transformed stiffness matrix $\mathbf{\bar{Q}}(\theta)$ using the formula provided:

$$\mathbf{\bar{Q}}(\theta) = \mathbf{T}^{-1}(\theta) \; \mathbf{Q}_\text{local} \; \mathbf{T}^{-T}(\theta)$$

Note: This is a matrix multiplication operation, and you can use the `@` operator for matrix multiplication in NumPy. If you were to us the * operator, it would perform element-wise multiplication.

```{note}
**Elementwise Multiplication**: Each corresponding element of two matrices (or arrays) is multiplied independently. This operation requires matrices of the same shape and results in a matrix of the same dimensions, where each element is the product of the corresponding elements. For example:

$$
\begin{bmatrix}
a & b \\
c & d
\end{bmatrix}
\odot
\begin{bmatrix}
e & f \\
g & h
\end{bmatrix}
=
\begin{bmatrix}
a \cdot e & b \cdot f \\
c \cdot g & d \cdot h
\end{bmatrix}
$$

---

**Matrix Multiplication**: This follows the linear algebra rules for multiplying matrices. The elements of the resulting matrix are computed as the dot product of rows from the first matrix with columns from the second. For example:

$$
\begin{bmatrix}
a & b \\
c & d
\end{bmatrix}
\cdot
\begin{bmatrix}
e & f \\
g & h
\end{bmatrix}
=
\begin{bmatrix}
a \cdot e + b \cdot g & a \cdot f + b \cdot h \\
c \cdot e + d \cdot g & c \cdot f + d \cdot h
\end{bmatrix}
$$

**Key Difference**: Elementwise multiplication is straightforward and independent for each element, while matrix multiplication involves summing products across rows and columns, capturing relationships between elements.
```

In [ ]:
def transform_stiffness(Q_local, theta, otter=False):
    """
    Transform Q_local into global coords by angle theta.
    Q_bar = T_inv * Q_local * T_inv^T
    """
    # 1. Compute the transformation matrix T
    # BEGIN SOLUTION
    T = transformation_matrix_2D(theta)
    # END SOLUTION
    
    # 2. Compute the inverse of the transformation matrix T
    # BEGIN SOLUTION
    T_inv = np.linalg.inv(T)
    # END SOLUTION
    
    # 3. Compute the transpose of the inverse of the transformation matrix T
    # BEGIN SOLUTION
    T_inv_transpose = T_inv.T
    # END SOLUTION
    
    # 4. Compute the transformed stiffness matrix Q_bar
    # BEGIN SOLUTION
    Q_bar = T_inv @ Q_local @ T_inv_transpose
    # END SOLUTION
    
    # Do not change the code below this line
    if otter:
        return T, T_inv, T_inv_transpose, Q_bar
    else:
        return Q_bar

In [ ]:
""" # BEGIN TEST CONFIG
points: 1
hidden: false
success_message: "Success: transform_stiffness Q_bar is 3x3!"
failure_message: "Failed: Q_bar is not 3x3!"
log_variables: ["Q_bar_shape"]
"""  # END TEST CONFIG

import numpy as np
from solution import transform_stiffness

# Test (3): Check the shape of Q_bar
Q_local = np.eye(3)
theta = 0.0
Q_bar = transform_stiffness(Q_local, theta)  # otter=False
Q_bar_shape = Q_bar.shape
assert Q_bar_shape == (3, 3), f"Expected Q_bar to be 3x3, but got {Q_bar_shape}."

In [ ]:
""" # BEGIN TEST CONFIG
points: 1
hidden: false
success_message: "Success: transform_stiffness with theta=0 keeps Q_local unchanged!"
failure_message: "Failed: Q_bar != Q_local for theta=0!"
log_variables: ["Q_local","Q_bar"]
"""  # END TEST CONFIG

import numpy as np
from solution import transform_stiffness

# Test (4): For theta=0, we expect Q_bar == Q_local
Q_local = np.array([[10.0, 2.0, 0.0], [2.0, 5.0, 1.0], [0.0, 1.0, 3.0]])
theta = 0.0
Q_bar = transform_stiffness(Q_local, theta)
assert np.allclose(
    Q_bar, Q_local, atol=1e-9
), f"Expected Q_bar == Q_local, but got:\n{Q_bar}"

In [ ]:
""" # BEGIN TEST CONFIG
points: 2
hidden: false
success_message: "Success: transform_stiffness returns T, T_inv, and T_inv_transpose consistent with each other!"
failure_message: "Failed: T * T_inv != Identity or T_inv.T != T_inv_transpose!"
log_variables: ["T_times_T_inv","I","T_inv_transpose","T_inv_T"]
"""  # END TEST CONFIG

import numpy as np

# Test (5): Check T * T_inv = I, and T_inv_transpose = T_inv.T
Q_local = np.eye(3)
theta = np.pi / 6
T, T_inv, T_inv_transpose, Q_bar = transform_stiffness(Q_local, theta, otter=True)

I = np.eye(3)
T_times_T_inv = T @ T_inv

# 5(a) T @ T_inv should be close to Identity
assert np.allclose(
    T_times_T_inv, I, atol=1e-9
), f"T * T_inv is not the identity.\nGot:\n{T_times_T_inv}"

# 5(b) T_inv_transpose should be T_inv.T
T_inv_T = T_inv.T
assert np.allclose(
    T_inv_transpose, T_inv_T, atol=1e-9
), f"T_inv_transpose != T_inv.T.\nGot:\n{T_inv_transpose}\nExpected:\n{T_inv_T}"

In [ ]:
""" # BEGIN TEST CONFIG
points: 2
hidden: false
success_message: "Success: transform_stiffness with random Q_local is dimensionally consistent!"
failure_message: "Failed: Q_bar has the wrong shape or incorrect values for a random Q_local!"
log_variables: ["Q_bar","Q_test"]
"""  # END TEST CONFIG

import numpy as np
import math

# Test (6): For a random Q_local, just check dimension & numeric stability
rng = np.random.default_rng(42)
Q_test = rng.random((3, 3))
theta = math.pi / 4

# We want Q_local to be symmetric (like a typical stiffness matrix),
# so let's force it to be symmetric for realism:
Q_test = 0.5 * (Q_test + Q_test.T)

Q_bar = transform_stiffness(Q_test, theta)
assert Q_bar.shape == (3, 3), f"Expected Q_bar to be 3x3, got shape {Q_bar.shape}"

# Optional advanced check: transformation is reversible if we rotate back by -theta
Q_bar_back = transform_stiffness(Q_bar, -theta)
assert Q_bar_back.shape == (
    3,
    3,
), f"Expected Q_bar_back to be 3x3, got shape {Q_bar_back.shape}"
# They won't be exactly the same due to numerical round-off, but let's do a close comparison
assert np.allclose(
    Q_bar_back, Q_test, atol=1e-9
), "After rotating Q_bar back by -theta, we did not get the original Q_test."

In [ ]:
""" # BEGIN TEST CONFIG
points: 3
hidden: false
success_message: "Success: transform_stiffness with random Q_local is dimensionally consistent!"
failure_message: "Failed: Q_bar has the wrong shape or incorrect values for a random Q_local!"
log_variables: ["Q_bar","Q_test"]
"""  # END TEST CONFIG

import numpy as np
import math

# Test (6): For a random Q_local, just check dimension & numeric stability
rng = np.random.default_rng(42)
Q_test = rng.random((3, 3))
theta = math.pi / 4

# We want Q_local to be symmetric (like a typical stiffness matrix),
# so let's force it to be symmetric for realism:
Q_test = 0.5 * (Q_test + Q_test.T)

Q_bar = transform_stiffness(Q_test, theta)
assert Q_bar.shape == (3, 3), f"Expected Q_bar to be 3x3, got shape {Q_bar.shape}"

# Optional advanced check: transformation is reversible if we rotate back by -theta
Q_bar_back = transform_stiffness(Q_bar, -theta)
assert Q_bar_back.shape == (
    3,
    3,
), f"Expected Q_bar_back to be 3x3, got shape {Q_bar_back.shape}"
# They won't be exactly the same due to numerical round-off, but let's do a close comparison
assert np.allclose(
    Q_bar_back, Q_test, atol=1e-9
), "After rotating Q_bar back by -theta, we did not get the original Q_test."

In [ ]:
""" # BEGIN TEST CONFIG
points: 2
hidden: false
success_message: "Success: transform_stiffness with Q_local=I and theta=π/2 is correct!"
failure_message: "Failed: transform_stiffness with Q_local=I, θ=π/2 is incorrect!"
log_variables: ["Q_local","Q_bar","Q_expected"]
"""  # END TEST CONFIG

import numpy as np
import math

# Test (7): If Q_local=I (3x3 identity) and we rotate by θ=π/2,
# then Q_bar = T_inv * I * T_inv^T = T_inv * T_inv^T.
# But T_inv = T^(-1). For a standard 2D transformation matrix from the previous function,
# rotating the identity in principal axes might still yield an identity, because
# in many classical transformation contexts, rotating an identity matrix is still an identity.
#
# We can simply check if Q_bar is still the identity. Let's verify numerically.

Q_local = np.eye(3)
theta = math.pi / 2
Q_bar = transform_stiffness(Q_local, theta)

Q_expected = np.eye(3)  # We expect the identity remains the identity
assert np.allclose(Q_bar, Q_expected, atol=1e-9), f"Expected Q_bar = I, got:\n{Q_bar}"

### Part 3: Effective Stiffness of Anisotropic Filler Distribution

Generally, composite materials are designed with a specific orientation distribution of the filler phase to achieve desired mechanical properties. In this context, we will consider an **anisotropic distribution** of filler orientations in the composite material. The filler phase is assumed to have a local stiffness matrix $\mathbf{Q}_\text{local}$ in its principal axes, and the orientation distribution is characterized by the volume fraction of fibers $\phi_i$ oriented at each angle $\theta$. You can compute the effective stiffness of the composite material by computing the sum over all orientations:

$$
\mathbf{Q}_\text{filler}^\text{(eff)} 
= \sum_{i=1}^{n} \phi_i \, \mathbf{\bar{Q}}(\theta_i),
\quad
\text{where}
\quad
\mathbf{\bar{Q}}(\theta_i)
= \text{transform\_stiffness}(\mathbf{Q}_\text{local}, \theta_i).
$$

### Python Implementation:

1. initialize a 3 by 3 zero array `Q_bar` using the `np.zeros` function. This array will store the sum of the transformed stiffness matrices for each orientation.

2. Inside the loop convert the angle in degrees to radians using the `np.radians` function and assign it to the variable `theta_rad`.

3. Compute the transformed stiffness matrix `transformed_stiffness` using the `transform_stiffness` function with the local stiffness matrix `Q_local` and the angle `theta_rad`.

4. Use the add assignment operator `+=` to add the transformed stiffness matrix `transformed_stiffness` to the `Q_bar` array, weighted by the volume fraction `phi_i`.

5. Test this code by computing the effective stiffness matrix `Q_filler_eff` for following information.

    5.1 Local Stiffness Matrix:

    Compute the local stiffness matrix `Q_local` using the `local_stiffness_matrix` function with the given elastic moduli and shear modulus.

    $E_1$ = 100 GPa
    $E_2$ = 10 GPa
    $\nu_{12}$ = 0.3  
    $G_{12}$ = 5 GPa

    5.2 Filler Orientations and Volume Fractions: Make the following lists for the angles and volume fractions.

    Angles - [0, 30, 60, 90]
    Volume Fractions - [0.12, 0.03, 0.02, 0.03]

    To do this, make lists `angles_deg` and volume fractions `phis` for the angles and volume fractions, respectively. 

    5.3 Compute the  effective filler stiffness 
    Compute the  effective filler stiffness by calling the `compute_filler_stiffness` function with the local stiffness matrix `Q_local`, angles in degrees `angles_deg`, and volume fractions `phis` as inputs. Assign the result to the variable `Q_filler_eff`.

In [ ]:
def compute_filler_stiffness(Q_local, angles_deg, phis):
    """
    Compute the stiffness matrix for a laminate with given fiber angles and volume fractions.
    """
    # 1. Initialize the stiffness matrix
    # BEGIN SOLUTION
    Q_bar = np.zeros((3, 3))
    # END SOLUTION

    # Do not modify this line of code, this code will iterate over the list of angles and phis one at a time
    # make sure that your code is tab indented inside the for loop
    for angle_deg, phi_i in zip(angles_deg, phis):

        # 2. Compute the angle in radians
        theta_rad = np.radians(angle_deg)  # SOLUTION

        # 3. Compute the transformed stiffness matrix Q_bar
        transformed_stiffness = transform_stiffness(Q_local, theta_rad)  # SOLUTION

        # 4. Update the global stiffness matrix Q_bar using the transformed stiffness matrix and the volume fraction
        Q_bar += phi_i * transformed_stiffness  # SOLUTION

    # Do not change the code below this line
    return Q_bar

# 5.1 Compute the local stiffness matrix
# BEGIN SOLUTION
Q_local = local_stiffness_orthotropic(100, 10, 0.3, 5)
# END SOLUTION

# 5.2 Assign the list of fiber angles and volume fractions
# BEGIN SOLUTION
angles_deg = [0, 30, 60, 90]
phis = [0.12, 0.03, 0.02, 0.03]
# END SOLUTION

# 5.3 Compute the global stiffness matrix
# BEGIN SOLUTION
Q_bar = compute_filler_stiffness(Q_local, angles_deg, phis)
# END SOLUTION

In [ ]:
Q_bar

In [ ]:
""" # BEGIN TEST CONFIG
points: 1
hidden: false
success_message: "Success: compute_filler_stiffness returns correct Q_bar shape!"
failure_message: "Failed: Q_bar does not have shape (3, 3)!"
log_variables: ["Q_bar_shape"]
"""  # END TEST CONFIG


Q_bar_shape = Q_local.shape
assert Q_bar_shape == (3, 3), f"Expected Q_bar to be (3, 3), but got {Q_bar_shape}."


In [ ]:
""" # BEGIN TEST CONFIG
points: 1
hidden: false
success_message: "Success: compute_filler_stiffness initializes Q_bar correctly!"
failure_message: "Failed: Q_bar is not initialized to a zero matrix!"
log_variables: ["Q_bar_initial"]
"""  # END TEST CONFIG

import numpy as np

angles_deg = []
phis = []
Q_bar = compute_filler_stiffness(Q_local, angles_deg, phis)
Q_bar_initial = np.zeros((3, 3))
assert np.allclose(
    Q_bar, Q_bar_initial, atol=1e-9
), "Q_bar should be initialized to a zero matrix."


In [ ]:
""" # BEGIN TEST CONFIG
points: 2
hidden: false
success_message: "Success: compute_filler_stiffness handles single angle and phi correctly!"
failure_message: "Failed: The function does not correctly compute Q_bar for a single angle and phi."
log_variables: ["Q_bar_expected", "Q_bar_computed"]
"""  # END TEST CONFIG

import numpy as np

# # Test (4): Single angle and phi
# Q_local_test = np.array([[100.0, 0.0, 0.0], [0.0, 10.0, 0.0], [0.0, 0.0, 5.0]])
# angles_deg_test = [45]
# phis_test = [1.0]

# # Compute expected Q_bar using transform_stiffness manually
# theta_rad_test = np.radians(45)


transformed_stiffness = transform_stiffness(Q_local_test, theta_rad_test)
Q_bar_expected = np.array([[32.5, 22.5, 22.5], [22.5, 32.5, 22.5], [22.5, 22.5, 27.5]])

Q_bar_computed = compute_filler_stiffness(Q_local_test, angles_deg_test, phis_test)
assert np.allclose(
    Q_bar_computed, Q_bar_expected, atol=1e-9
), f"Expected Q_bar:\n{Q_bar_expected}\nBut got:\n{Q_bar_computed}"

In [ ]:
""" # BEGIN TEST CONFIG
points: 1
hidden: false
success_message: "Success: compute_filler_stiffness handles multiple angles and phis correctly!"
failure_message: "Failed: The function does not correctly compute Q_bar for multiple angles and phis."
log_variables: ["Q_bar_expected", "Q_bar_computed"]
"""  # END TEST CONFIG

import numpy as np

# Test (5): Multiple angles and phis
Q_local_test = np.array([[50.0, 50.0, 0.0], [27.0, 10.0, 0.0], [0.0, 0.0, 5.0]])
angles_deg_test = [0, 30, 60, 90]
phis_test = [0.12, 0.03, 0.02, 0.03]

Q_bar_expected = np.array(
    [
        [8.246875, 8.445625, 0.14397672],
        [6.260625, 4.446875, 0.22408407],
        [0.64194133, 0.72204868, 0.653125],
    ]
)

# Compute Q_bar using the function
Q_bar_computed = compute_filler_stiffness(Q_local_test, angles_deg_test, phis_test)
assert np.allclose(
    Q_bar_computed, Q_bar_expected, atol=1e-9
), f"Expected Q_bar:\n{Q_bar_expected}\nBut got:\n{Q_bar_computed}"

In [ ]:
""" # BEGIN TEST CONFIG
points: 5
hidden: false
success_message: "Success: compute_filler_stiffness handles multiple angles and phis correctly!"
failure_message: "Failed: The function does not correctly compute Q_bar for multiple angles and phis."
log_variables: ["Q_bar_expected", "Q_bar_computed"]
"""  # END TEST CONFIG

import numpy as np


Q_bar_expected = np.array(
    [
        [14.61736882, 1.40180373, 1.07508184],
        [1.40180373, 5.98971998, 0.8911716],
        [1.07508184, 0.8911716, 1.79635469],
    ]
)

# Compute Q_bar using the function
Q_bar_computed = compute_filler_stiffness(Q_local_test, angles_deg_test, phis_test)
assert np.allclose(
    Q_bar, Q_bar_expected, atol=1e-9
), f"Expected Q_bar:\n{Q_bar_expected}\nBut got:\n{Q_bar}"

## Part 4: Effective Stiffness of Composite Material

Now we need to add the mechanical properties of the matrix to determine the ensemble properties of the composite material. The effective stiffness matrix $\mathbf{Q}_\text{eff}$ of the composite material can be computed by adding the stiffness matrix of the matrix phase $\mathbf{Q}_\text{matrix}$ to the effective stiffness matrix of the filler phase $\mathbf{Q}_\text{filler}^\text{(eff)}$ weighted by their effective volume fractions.

Python Implementation:

1. Compute the fraction of the total composite composed of filler by taking the sum of the volume fractions `phis` and assign it to the variable `f_filler`. You should use the `np.sum` function to compute the sum of the volume fractions.

2. compute the effective stiffness matrix `Q_eff` by adding the matrix stiffness `Q_matrix` to the effective filler stiffness `Q_bar` weighted by the fraction of filler `f_filler`. The formula for computing the effective stiffness matrix is given by:

$$\mathbf{Q}_\text{eff} = (1 - f_\text{filler}) \cdot \mathbf{Q}_\text{matrix} +  \mathbf{Q}_\text{bar}$$

In [ ]:
# Combine with matrix

# Compute the sum of the volume fractions
# BEGIN SOLUTION
f_filler = np.sum(phis)  
# END SOLUTION

# Compute the effective stiffness matrix
# BEGIN SOLUTION
Q_eff = (1 - f_filler) * Q_matrix + f_filler * Q_bar
# END SOLUTION

# We have provided the code to print the results for you
print("Local Filler Q:\n", Q_local, "\n")
print("Matrix Q:\n", Q_matrix, "\n")
print("Effective Filler Q:\n", Q_bar, "\n")
print("Overall Effective Q:\n", Q_eff)

In [ ]:
Q_eff

In [ ]:
""" # BEGIN TEST CONFIG
points: 3
hidden: false
success_message: "Sucess: The filler volume fraction is computed correctly!"
failure_message: "Failed: The filler volume fraction is incorrect!"
log_variables: ["f_filler"]
"""  # END TEST CONFIG

import numpy as np

f_filler_exp = 0.19999999999999998

In [ ]:
""" # BEGIN TEST CONFIG
points: 3
hidden: false
success_message: "Success: compute_filler_stiffness handles multiple angles and phis correctly!"
failure_message: "Failed: The function does not correctly compute Q_bar for multiple angles and phis."
log_variables: ["Q_bar_expected"]
"""  # END TEST CONFIG

import numpy as np


Q_bar_expected = np.array(
    [
        [4.71900737, 0.87288684, 0.21501637],
        [0.87288684, 2.99347761, 0.17823432],
        [0.21501637, 0.17823432, 0.99927094],
    ]
)

assert np.allclose(
    Q_eff, Q_bar_expected, atol=1e-5
), f"Expected Q_bar:\n{Q_bar_expected}\nBut got:\n{Q_bar}"

## Visualization

We have provided some visualizations to help you understand your results. You can run the code cell below to visualize the local stiffness matrix, the transformed stiffness matrix for different orientations, and the effective stiffness matrix of the composite material.

In [ ]:
from composite_viz import *

# polar plot of stiffness vs orientation
plot_stiffness_polar(transform_stiffness, Q_local)

In [ ]:
# Volume fraction for different fiber orientations
plot_volume_fractions(transform_stiffness, angles_deg, phis)

In [ ]:
# Stiffness matrix
plot_stiffness_matrix(Q_eff, title="Effective Stiffness Matrix")

In [ ]:
# Stiffness vs orientation linear
plot_stiffness_line(Q_local)